In [12]:
import os
from collections import Counter
import multiprocessing
import json
import regex as re

In [ ]:
multiprocessing.cpu_count()

96

In [2]:
!unzip '/content/sample_data/tokenizer_training/train.zip'

Streaming output truncated to the last 5000 lines.
  inflating: train/143913.txt        
  inflating: train/143914.txt        
  inflating: train/143915.txt        
  inflating: train/143916.txt        
  inflating: train/143919.txt        
  inflating: train/14392.txt         
  inflating: train/143920.txt        
  inflating: train/143921.txt        
  inflating: train/143922.txt        
  inflating: train/143923.txt        
  inflating: train/143924.txt        
  inflating: train/143925.txt        
  inflating: train/143926.txt        
  inflating: train/143927.txt        
  inflating: train/143928.txt        
  inflating: train/14393.txt         
  inflating: train/143930.txt        
  inflating: train/143931.txt        
  inflating: train/143932.txt        
 extracting: train/143933.txt        
  inflating: train/143934.txt        
  inflating: train/143935.txt        
  inflating: train/143937.txt        
  inflating: train/143938.txt        
  inflating: train/14394.txt         

In [3]:
import os
from pathlib import Path
import glob

def read_text_files(folder_path):
    # Method 1: Using glob
    all_text = ""
    file_count = 0

    # Get all .txt files in the folder
    txt_files = glob.glob(os.path.join(folder_path, "*.txt"))
    total_files = len(txt_files)

    print(f"Found {total_files} text files")

    for file_path in txt_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                all_text += file.read() + " "  # Add space between files
                file_count += 1

            # Print progress every 1000 files
            if file_count % 1000 == 0:
                print(f"Processed {file_count}/{total_files} files")

        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")

    print(f"\nSuccessfully read {file_count} files")
    print(f"Total characters: {len(all_text)}")

    return all_text

In [4]:
text = read_text_files(r'/content/train')

Found 44000 text files
Processed 1000/44000 files
Processed 2000/44000 files
Processed 3000/44000 files
Processed 4000/44000 files
Processed 5000/44000 files
Processed 6000/44000 files
Processed 7000/44000 files
Processed 8000/44000 files
Processed 9000/44000 files
Processed 10000/44000 files
Processed 11000/44000 files
Processed 12000/44000 files
Processed 13000/44000 files
Processed 14000/44000 files
Processed 15000/44000 files
Processed 16000/44000 files
Processed 17000/44000 files
Processed 18000/44000 files
Processed 19000/44000 files
Processed 20000/44000 files
Processed 21000/44000 files
Processed 22000/44000 files
Processed 23000/44000 files
Processed 24000/44000 files
Processed 25000/44000 files
Processed 26000/44000 files
Processed 27000/44000 files
Processed 28000/44000 files
Processed 29000/44000 files
Processed 30000/44000 files
Processed 31000/44000 files
Processed 32000/44000 files
Processed 33000/44000 files
Processed 34000/44000 files
Processed 35000/44000 files
Proces

In [5]:
def _clean_text(text: str) -> str:
    """
    Clean text to keep ONLY Hindi characters.
    Removes:
    - Emojis
    - English characters
    - Other scripts
    - Punctuation
    - Extra whitespace
    """

    # Step 1: Remove emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub('', text)

    # Step 2: Remove URLs
    text = re.sub(r'http\S+|www.\S+', '', text)

    # Step 3: Keep ONLY Devanagari and spaces
    # \u0900-\u097F is Devanagari range
    allowed_pattern = re.compile(r'[^\u0900-\u097F\s]')
    text = allowed_pattern.sub('', text)

    # Step 4: Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [6]:
#cleaned_text_tokens = list(_clean_text(text).encode('utf-8'))
cleaned_text = _clean_text(text)

In [7]:
#Taking first 10M characters
cleaned_text_subset = cleaned_text[:10000000]
len(cleaned_text_subset)

10000000

In [10]:
def _get_pairs(tokens):
  return Counter(zip(tokens, tokens[1:]))

In [ ]:
stats = _get_pairs(cleaned_text)
print(f"total number of pairs is {len(stats)}")

total number of pairs is 5463


In [14]:
def _merge(input_chars, pair, new_token):
  new_tokens = []
  #print(len(input_chars))
  i = 0
  while i < len(input_chars):
    if i < len(input_chars)-1 and input_chars[i] == pair[0] and input_chars[i+1] == pair[1]:
      new_tokens.append(new_token)
      i += 2
    else:
      new_tokens.append(input_chars[i])
      i += 1
  return new_tokens


In [ ]:
##Change logs
# Vocab_size = 5
# 1. Input was raw bytes. time taken = 300 secs
# 2. Input was changed to characters instead of bytes. Time take = 177 secs
# 3. Tried multiprocessing with 6/20 processes but its actually taking more time!!!.
      #The overhead of spawning new processes is a overkill
# 4. Shall i try this in rust?

In [10]:
def _train_tokenizer(inputs_tokens, vocab_size = 1):
  ids = list(inputs_tokens)
  merges = {}
  for i in range(vocab_size):
    stats = _get_pairs(ids)
    pair = max(stats, key = stats.get)
    new_token = 256+ i
    ids = _merge(ids, pair, new_token)
    print(f"Merging tokens {pair} -> {new_token}")
    merges[pair] = new_token

    #Saving merges in a json format
    if i%200 ==0:
      # Convert tuples to strings only during saving
      print(f"Percentage completion: {(i/vocab_size)*100:.2f}%")
      json_merges = {f"{k[0]},{k[1]}": v for k, v in merges.items()}
      with open(f'merges_{i}.json', 'w') as f:
          json.dump(json_merges, f)

  return merges, ids



In [ ]:
def _process_chunk(chunk, pair, new_token):
    """Process a single chunk of the data"""
    print(os.getpid())
    return _merge(chunk, pair, new_token)

def _train_tokenizer(inputs_tokens, vocab_size = 1, num_processes = 2):
  ids = list(inputs_tokens)
  merges = {}
  for i in range(vocab_size):
    stats = _get_pairs(ids)
    pair = max(stats, key = stats.get)
    new_token = 256+ i
    with Pool(processes = num_processes) as pool:
      chunk_size = len(ids) // num_processes
      chunks = [ids[j:j + chunk_size] for j in range(0, len(ids), chunk_size)]
      merge_func = partial(_process_chunk, pair=pair, new_token=new_token)
      merged_chunks = pool.map(merge_func, chunks)
      for chunk in merged_chunks:
        ids.extend(chunk)

      print(f"Merging tokens {pair} -> {new_token}")
      merges[pair] = new_token

  return merges

In [11]:
tokens = cleaned_text_subset.encode('utf-8')

In [12]:
vocab_json, new_ids = _train_tokenizer(tokens, 4500)

Merging tokens (224, 164) -> 256
Percentage completion: 0.00%
Merging tokens (32, 256) -> 257
Merging tokens (224, 165) -> 258
Merging tokens (256, 190) -> 259
Merging tokens (256, 176) -> 260
Merging tokens (258, 141) -> 261
Merging tokens (258, 135) -> 262
Merging tokens (261, 256) -> 263
Merging tokens (259, 256) -> 264
Merging tokens (256, 191) -> 265
Merging tokens (257, 149) -> 266
Merging tokens (265, 256) -> 267
Merging tokens (258, 128) -> 268
Merging tokens (256, 130) -> 269
Merging tokens (262, 257) -> 270
Merging tokens (259, 257) -> 271
Merging tokens (258, 139) -> 272
Merging tokens (256, 168) -> 273
Merging tokens (256, 149) -> 274
Merging tokens (268, 257) -> 275
Merging tokens (256, 164) -> 276
Merging tokens (260, 257) -> 277
Merging tokens (258, 136) -> 278
Merging tokens (269, 257) -> 279
Merging tokens (256, 184) -> 280
Merging tokens (164, 257) -> 281
Merging tokens (258, 129) -> 282
Merging tokens (261, 260) -> 283
Merging tokens (256, 178) -> 284
Merging tokens 

In [13]:
len(new_ids)

2883630

In [14]:
len(tokens)/len(new_ids)

9.099939312602519

In [ ]:
#Compressed IDs : 2632695
#Compression ration: 3.79X
#Time take: 5 Hrs 40 Minutes


**Encoding**

In [2]:
#load the vocab_json
import json
with open('/content/sample_data/merges_final (1).json', 'r') as f:
  merges = json.load(f)

In [3]:
new_merges = {}
for k, v in merges.items():
  try:
    tuple_value = tuple(map(int, k.split(',')))
    new_merges[tuple_value] = v
  except:
    print("Key:" , k)
    print("Value:", v)

In [ ]:
def merge_components(component):
    """Recursively merge components into a single string"""
    if isinstance(component, str):
        return component
    if isinstance(component, tuple):
        return ''.join(merge_components(item) for item in component)
    return component

def merge_pair_components(d):
    """Merge elements within each component of the pair while maintaining pair structure"""
    merged_dict = {}

    for key, value in d.items():
        if not isinstance(value, tuple):
            merged_dict[key] = value
            continue

        # Handle the pair
        if len(value) == 2:
            c1, c2 = value

            # Recursively merge components
            c1 = merge_components(c1)
            c2 = merge_components(c2)

            merged_dict[key] = (c1, c2)
        else:
            merged_dict[key] = value

    return merged_dict

# Example usage:
merged_vocab = merge_pair_components(vocab)

**Encoding**

In [4]:
def encode(input):
    tokens = list(input.encode("utf-8"))
    while True:
        stats = _get_pairs(tokens)
        pair = min(stats, key = lambda p: new_merges.get(p, float("inf")))
        if pair not in new_merges:
            break #Nothing else can be merged
        idx = new_merges[pair]
        tokens = _merge(tokens, pair, idx)
    return tokens

Decoding

In [7]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in new_merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

print(decode([334,2056]))

 हातु


In [8]:
text = 'नई दिल्ली: भारतीय क्रिकेट टीम के पूर्व दिग्गज स्पिन'

In [16]:
print((encode(text)))

[273, 421, 2649, 268, 58, 1527, 2656, 3069, 3210, 359, 952, 1933, 344, 2789, 346]


In [ ]:
val_text = "निर्देशांक: 25°27′N 81°51′E / 25.45°N 81.85°E / 25.45; 81.85 सराय जोधपुरी फूलपुर, इलाहाबाद, उत्तर प्रदेश स्थित एक गाँव है।"

In [ ]:
val_text1 = """राजस्थान की राजधानी जयपुर को गुलाबी नगरी नाम से भी जाना जाता है। इस नगर में सुंदर नगरों, हवेलियों और किलों की भरमार है। जयपुर का अर्थ है - जीत का नगर। इसे कुशवाह राजपूत राजा सवाई जय सिंह, द्वितीय ने 1727 में बसाया था। उस समय का यह प्रथम नगर था जो योजनाबद्ध ढ़ंग से बसाया गया था। इसका ख़ाका तैयार किया था वास्तु शिल्पी विद्याधर भट्टाचार्य ने।
मुगलों ने जो भवन बनवाए उनमें लाल पत्थर का उपयोग किया जबकि राजा जय सिंह ने पूरे नगर को गुलाबी रंग से पुतवाया। जिसके कारण जयपुर को गुलाबी नगर कहा जाता है l
जयपुर को यदि पास से देखना हो तो पूरे नगर को पैरों से नापना एक अच्छा विकल्प हो सकता है। राष्ट्रीय राजधानी दिल्ली से यह नगर 262 किमी की दूरी पर स्थित है और बस, रेल और हवाई यातायात से अच्छे से जुड़ा है। यहाँ की प्रमुख भाषाएँ हिन्दी और राजस्थानी है।
जयपुर में आने के बाद पता चलता है, कि आप किसी रजवाडे में प्रवेश कर गये है हैं, शाही साफ़ा बांधे जयपुर के बना और लंहगा-चुन्नी से सजी जयपुर की नारियां, गपशप मारते जयपुर के वृद्ध लोग, राजस्थानी भाषा में कितनी प्यारी बोलियां,पधारो म्हारे देश जैसा स्वागत और बैठो सा, जीमो सा, जैसी बातें, कितनी सुहावनी लगती है।
यहाँ के विभिन्न दर्शनीय स्थल इस प्रकार हैं।
सिटी पैलेस जयपुर का एक प्रमुख लैंडमार्क है।राजस्थानी व मुगल शैलियों की मिश्रित रचना एक पूर्व शाही निवास जो पुराने शहर के बीचों बीच खड़ा है। भूरे संगमरमर के स्तंभों पर टिके नक्काशीदार मेहराब, सोने व रंगीन पत्थरों की फूलों वाली आकृतियों ले अलंकृत है। संगमरमर के दो नक्काशीदार हाथी प्रवेश द्वार पर प्रहरी की तरह खड़े है। जिन परिवारों ने पीढ़ी दर पीढ़ी राजाओं की सेवा की है वे लोग गाइड के रूप में कार्य करते है। पैलेस में एक संग्राहलय है जिसमें राजस्थानी पोशाकों व मुगलों तथा राजपूतों के हथियार का बढ़िया संग्रह हैं। इसमें विभिन्न रंगों व आकारों वाली तराशी हुई मूंठ वाकी तलवारें भी हैं, जिनमें से कई मीनाकारी के जड़ऊ काम व जवाहरातों से अलंकृत है तथा शानदार जड़ी हुई म्यानों से युक्त हैं। महल में एक कलादीर्घा भी हैं जिसमें लघुचित्रों, कालीनों, शाही साजों सामान और अरबी, फारसी, लेटिन व संस्कृत में दुर्लभ खगोल विज्ञान की रचनाओं का उत्कृष्ट संग्रह है जो सवाई जयसिंह द्वितीय ने विस्तृत रूप से खगोल विज्ञान का अध्ययन करने के लिए प्राप्त की थी।
सिटी पैलेस से कुछ दूरी पर स्थित है, जयपुर का जंतर मंतर। यह एक पत्थर की वेगशाला है। यह जयसिंह की पाँच वेगशालाओं में से सबसे विशाल है। इसके जटिल यंत्र, इसका विन्यास व आकार वैज्ञानिक ढंग से तैयार किया गया है। यह मध्ययुगीन भारत के खगोल विज्ञान के उच्च सूत्रों का प्रतिनिधित्व करतें है। इनमें सबसे प्रभावशाली रामयंत्र है जिसका उपयोग ऊंचाई नापने के लिए किया जाता है। जंतर मंतर परिसर में सवाई जय सिंह द्वारा तारों की गणना करने में उपयोग करने वाले कई उपकरण रखें हैं।
ईसवी सन् 1799 में निर्मित हवा महल राजपूतो का मुख्य प्रमाण चिन्ह है। यह जयपुर की पहचान भी है। पुरानी नगरी की मुख्य गलियों के साथ यह पाँच तलीय भवन गुलाबी रंग में अर्धअष्टभुजाकार और परिष्कृत छतेदार बलुए पत्थर की खिड़कीयों से सुसज्जित है। शाही घरानों की स्त्रियां नगर का दैनिक जीवन व नगर के जुलूस देख सके इसी उद्देश्य से इस भवन की रचना की गई थी। यहाँ से नगर का अच्छ दृश्य दिखता है। इस महल को इस प्रकार बनाया गया है, की अंदर से बाहर तो देखा जा सकता है, लेकिन बाहर से अंदर नहीं।
भगवान कृष्ण का जयपुर का सबसे प्रसिद्ध बिना शिखर का मंदिर। यह चन्द्र महल के पूर्व में बने जन निवास बगीचे के मध्य अहाते में स्थित है। संरक्षक देवता गोविंदजी की मुर्ति पहले वृंदावन के मंदिर में स्थापित थी जिसको सवाई जय सिंह द्वितीय ने अपने परिवार के देवता के रूप में यहाँ पुनः स्थापित किया था।
 - त्रिपोलिया बाजार के पशिचमी किनारे पर क्षितिज को छूती उच्चतम इमारत जिसका निर्माण ईसवी सन् 1749 में सवाई ईश्वरी सिंह मे महान विजय को मनाने के लिए किया था।
एक चिड़ियाघर, दरबा, पौधाघर, वनस्पति संग्राहलय से युक्त एक हरा भरा विस्तृत बाग, खेल का प्रसिद्ध मैदान भी है। बाढ राहत परियोजना के अंतर्गत ईसवी सन् 1865 में सवाई राम सिंह द्वितीय ने इसे बनवाया था। सर सवीटन जैकब द्वारा रूपांकित, अल्बर्ट हाल जो भारतीय वास्तुकला शैली का परिष्कृत नमूना है, जिसे बाद में उत्कृष्ट मूर्तियों, चित्रों, सज्जित बर्तनों, प्राकृतिक विज्ञान के नमूनों, इजिप्ट की एक ममी और प्रख्यात फारस के कालीनों से सुसज्जित कर खोला गया। हाल ही में, सांस्कृतिक कार्यक्रमों को बढ़ावा देने के लिए एक प्रेक्षागृह के साथ रवीन्द्र मंच, एक आधुनिक कलादीर्घा व एक खुला थियेटर इसमें बनाया गया हैं।
 - पुलिस स्मारक के पास गूंगे बहरों के विद्यालय के अहाते में विभिन्न देशों की प्यारी गुड़ियाँ यहाँ प्रदर्शित हैं।
- अपने आधुनिक कम्पयूटर युक्त प्रक्षेपण व्यवस्था के साथ इस ताराघर में श्रव्य व दृश्य शिक्षा व मनोरंजनों के साधनों की अनेखी सुविधा उपलब्घ है। विद्यालयों के दलों के लिये रियायत उपलब्ध है। प्रत्येक महीने के आखिरी बुघवार को यह बंद रहता है।
एक प्राचीन तार्थस्थल, निचली पहाड़ियों के बीच बगीचों से परे स्थित। मंदिर, मंडप और पवित्र कुंडो के साथ हरियाली युक्त प्राकृतिक दृश्य इसे आन्नददायक स्थल बना देते हैं। दीवान कृपाराम द्वारा निर्मित उच्चतम चोटी के शिखर पर बना सूर्य देवता का छोटा मंदिर शहर के सारे स्थानों से दिखाई पड़ता है।
आगरा मार्ग पर बने इस उत्कृष्ट जैन मंदिर की दीवारों पर जयपुर शैली में उन्नीसवीं सदी के अत्यधिक सुंदर चित्र बने हैं।Galta Ji Lake
मोती डूंगरी एक निजी पहाड़ी ऊचाँई पर बना किला है जो स्कॉटलैण्ड के महल की तरह निर्मित है। कुछ वर्षों पहले, पहाड़ी पादगिरी पर बना गणेश मंदिर और अद्भुत लक्ष्मी नारायण मंदिर भी उल्लेखनीय है।
चक्कर के मध्य सवाई जयसिंह का स्टैच्यू बहुत ही उत्कृष्ट ढंग से बना हुआ है। इसे जयपुर के संस्थापक को श्रद्धांजलि देने के लिए नई क्षेत्रीय योजना के अंतर्गत बनाया गया है।
आमेर मार्ग पर रामगढ़ मार्ग के चौराहे के पास रानियों की याद में बनी आकर्षक महारानी की छतरी है। मानसागर झील के मध्य, सवाई माधोसिंह प्रथम द्वारा निर्मित जल महल, एक वशीभूत करने वाला स्थल है। परिष्कृत मंदिरों व बगीचों वाले कनक वृंदावन भवन की पुरातन पूर्णता को अभी हाल ही में पुनर्निर्मित किया गया है। इस सड़क के पश्चिम में गैटोर में शाही शमशान घाट है जिसमें जयपुर के सवाई ईश्वरी सिंह के सिवाय समस्त शासकों के भव्य स्मारक हैं। बारीक नक्काशी व लालित्यपूर्ण आकार से युक्त सवाई जयसिंह द्वितीय की बहुत ही प्रभावशाली छतरी है। प्राकृतिक पृष्ठभूमि से युक्त बगीचे आगरा मार्ग पर दीवारों से घिरे शहर के दक्षिण पूर्वी कोने पर घाटी में फैले हुए हैं।
सिसोदिया रानी के बाग में फव्वारों, पानी की नहरों, व चित्रित मंडपों के साथ पंक्तिबद्ध बहुस्तरीय बगीचे हैं व बैठकों के कमरे हैं। अन्य बगीचों में, विद्याधर का बाग बहुत ही अच्छे ढ़ग से संरक्षित बाग है, इसमें घने वृक्ष, बहता पानी व खुले मंडप हैं। इसे शहर के नियोजक विद्याधर ने निर्मित किया था।
यह कभी सात शताव्दियों तक ढूंडार के पुराने राज्य के कच्छवाहा शासकों की राजधानी थी। आरंभिक ढ़ाचा अब थोड़ा ही बचा है।
लगभग दो शताब्दी पूर्व राजा मान सिंह, मिर्जा राजा जय सिंह और सवाई जय सिंह द्वारा निर्मित महलों, मंडपों, बगीचों और मंदिरों का एक आकर्षक भवन है। मावठा झील के शान्त पानी से यह महल सीधा उभरता है और वहाँ कठिन रास्ते द्वारा ही पहुंचा जा सकता है। सिंह पोल और जलेब चौक तक अकसर पर्यटक हाथी पर सवार होकर जाते हैं। चौक के सिरे से सीढ़ियों की पंक्तियाँ उठती हैं, एक शिला माता के मंदिर की ओर जाती है और दूसरी महल के भवन की ओर। यहां स्थापित करने के लिए राजा मान सिंह द्वारा संरक्षक देवी की मूर्ति, जिसकी पूजा हजारों श्रद्धालु करते है, पूर्वी बंगाल के जेसोर से यहां लाई गई थी। एक दर्शनीय खंभो वाला हॉल दीवान-ए-आम और एक दोमंजिला चित्रित प्रवेशद्वार, गणेश पोल आगे के पंरागण में है। गलियारे के पीछे चारबाग की तरह का एक रमणीय छोटा बगीचा है जिसकी दाई तरफ सुख निवास है और बाई तरफ जसमंदिर। इसमें मुगल व राजपूत वास्तुकला का मिश्रित है, बारीक ढंग से नक्काशी की हुई जाली की चिलमन, बारीक शीशों और गचकारी का कार्य और चित्रित व नक्काशीदार निचली दिवारें। मावठा झील के मध्य में सही अनुपातित मोहन बाड़ी या केसर क्यारी और उसके पूर्वी किनारे पर दिलराम बाग ऊपर बने महलों का मनोहर दृश्य दिखाते है।
कभी राजाओं, हस्तशिल्पों व आम जनता का आवास आमेर शहर अब खंडहर बन गया है। आकर्षक ढंग से नक्काशी किया व सुनियोजित जगत शिरोमणि मंदिर, मीराबाई से जुड़ा एक कृष्ण मंदिर, नरसिंहजी का पुराना मंदिर व अच्छे ढ़ग से बना सीढ़ियों वाला कुआँ, पन्ना मियां का कुण्ड समृद्ध अतीत के अवशेष है।
मध्ययुगीन भारत के कुछ सैनिक इमारतों में से एक। महलों, बगीचों, टांकियों, अन्य भन्डार, शस्त्रागार, एक सुनोयोजित तोप ढलाई-घर, अनेक मंदिर, एक लंबा बुर्ज और एक विशालकाय चढी हुई तोप - जय बाण जो देश की सबसे बड़ी तोपों में से एक है, इसमें अपना प्राचीन वैभव सुरक्षित करके रखा हुआ है। जयगढ़ के फैले हुए परकोटे, बुर्ज और प्रवेश द्वार पश्चिमी द्वार क्षितिज को छूते है। नाहरगढः जयगढ की पहाड़ियों के पीछे स्थित गुलाबी शहर का पहरेदार है - नाहरगढ़ किला। यद्यपि इसका बहुत कुछ हिस्सा ध्वस्त हो गया है, फिर भी सवाई मान सिंह द्वितीय व सवाई माधव सिंह द्वितीय द्वारा बनाई मनोहर इमारतें किले की रौनक बढ़ती है।
 - यह टोंक मार्ग पर स्थित है। इसके ध्वस्त महलों के अतिरिक्त, सांगानेर के उत्कृष्ट नक्काशीदार जैन मंदिर है। दो त्रिपोलिया के अवशेषो द्वारा नगर में प्रवेश किया जाता है।शिल्प उद्योग के लिए शहर महत्वपूर्ण केन्द्र है और ठप्पे व जालीदार छपाई की इकाइयों द्वारा हाथ से बने बढिया कपड़े यहां बनते है। यह कपड़ा देश व विदेश में प्रसिद्ध है।
 - अजमेर मार्ग पर, भूमि के छूता किला, अभी भी अच्छी अवस्था में है। यह अपने हाथ की छपाई के हथकरघा उद्योग के लिए उल्लेखनीय है, जहां सरल तकनीको का प्रयोग होता है। इस हथकरघाओं के डिजाइन कम जटिल व मटियाले रंगो के होते है।
 - पेड़ो से आच्छादित पहाड़ियो के बीच एक ऊंचा बांध बांधकर एक विशाल कृत्रिम झील की निर्माण किया गया है। यद्यपि जमवा माता का मंदिर व पुराने किले के खंडहर इसके पुरावशेष है। विशेषकर बारिश के मौसम में इसके आकर्षक प्राकृतिक दृश्य इसको एक बेहतर पिकनिक स्थल बना देते है।
 - सुन्दर सामोद महल का पुनर्निमाण किया गया तथा यह राजपूत हवेली वास्तुकला का बेहतर नमुना है व घूमने के लिए उत्तम स्थल।
 - खुदाई करने पर निकले एक वृताकार बौध मंदिर के अवशेषों से युक्त एक महत्वपूर्ण ऐतिहासिक स्थान है जो राजस्थान का असाधारण व भारत का आरंभिक प्रसिद्ध मंदिर है। बैराठ के मौर्य, मुगल व राजपूत समय के स्मृतिचिन्ह भी है। अकबर द्वारा निर्मित एक खान, एक रमणीय मुगल बगीचा औक जहांगीर द्वारा निर्मित चित्रित छतरियों व दीवारों से युक्त असाधारण इमारत अन्य आकर्षण है।
 - विशाल टापू वाली भारत की नमक की झील, पवित्र देवयानी कुंड, महल और पास ही स्थित नालियासार के प्रसिद्ध है।
 - मीणा कबीले के इस आवास में एक दुर्जय किला, एक जैन मंदिर और हरे भरे वातावरण के बीच एक बावड़ी है।
तुंगा - जयपुर व मराठा सेना के बीच हुए एतिहासिक युग का तुंगा गवाह है। सुंदर आम के बागो के बीच यह किला बसा है। जयपुर में बस्सी भी एक खास है यह एक छोटे जयपुर की तरह है। यहाँ पर कई आधिकारिक विभागों के अधिकारी और कर्मी जनता की समस्याओं को कैम्प अदालत के रूप में सुनकर उसका समाधान प्रस्तुत करने का प्रयास करते हैं। """

In [ ]:
val_text3= """इमारती पत्थरों को खोदकर निकालने की क्रिया को आखनन कहते हैं। इस स्थान को जहाँ से पत्थर निकाले जाते हैं, खादान या 'पाषाण खान' कहते हैं। पाषाण खान साधारणतया खुले स्थान में ही बनाई जाती है।
इमारती पत्थरों में ग्रैनाइट, बैसाल्ट, बालू के पत्थर, चूने के पत्थर, स्लेट और संगमरमर मुख्य हैं। ग्रैनाइट शब्द के अंतर्गत साधारणतया हल्के रंग की सभी आग्नेय शिलाएँ मानी जाती हैं। इन शिलाओं की रचना क्वार्ट्ज़, फ़ेल्स्पार, अभ्रक और हॉर्न ब्लेंड नामक खनिजों से होती है।
बैसाल्ट प्राय: काले रंग की शिलाएँ होती हैं। ये ट्रैप भी कहलाती हैं। इनमें फ़ेल्स्पार और पाइरॉक्सीन खनिजों की प्रचुर मात्रा होती है। इन शिलाओं में कई प्रकार के भंग होते हैं, जिनसे इन्हें खोदने में सुविधा होती है। ये सामान्यत: कड़ी होती हैं। ग्रैनाइट शब्द के अंतर्गत ही 'नाइस' नामक कायांतरित शिलाओं को भी गिन लिया जाता है। अभ्रक आदि खनिज के समांतर तलों में व्यवस्थित होने से इनमें अनेक दुर्बल धरातल बन जाते हैं, जिनके कारण इन्हें खोदने में सुकरता हो जाती है। भंगों की उपस्थिति में इसे और भी सरलता से खोदा जा सकता है। बालुकाश्म एवं चूने का पत्थर जलज शिलाएँ हैं। अत: इनमें स्वाभाविक रूप से स्तर होते हैं। स्तरों की उपस्थिति के कारण इनका खोदना और इन्हें सिल्लियों का रूप देना अत्यंत सरल हो जाता है। कायांतरण के प्रभाव से चूने के पत्थर संगमरमर की शिलाओं में परिवर्तित हो जाते हैं, परंतु उनकी स्तररचना नष्ट हो जाती है। संगमरमर की शिलाओं को तोड़ने के लिए भंगों का सहारा लेना पड़ता है। स्लेट भी कायांतरित शिला है। इसमें समांतर तड़कन होती है, अत: इसकी अत्यंत पतली परतें निकाली जा सकती हैं।
किसी भी पत्थर को खोद निकालने के पूर्व उसकी कठोरता, शक्ति, खनिज रचना, रध्रंता और चिकना करने पर प्राप्त और सुंदरता की परीक्षा की जाती है। खोदने के स्थान पर पत्थरों में अत्यधिक भंग, दरार अथवा ऐसे अन्य दुर्बल धरातल नहीं होने चाहिए जिनसे पुष्ट और बड़ी सल्लियाँ न मिल सकें, परंतु यदि ऐसे धरातल हों ही नहीं तो भी कठिनाई पड़ेगी। तब खोदे हुए पत्थरों को चारों ओर से घिसने का व्यय बढ़ा जाएगा। पत्थरों में अत्यधिक तथा अनियमित अपक्षय भी नहीं होना चाहिए।
पत्थरों की कठोरता, दुर्बल धरातलों की उपस्थिति, सिल्लियों की माप और खदान की विस्तृति पर खोदने की क्रिया का निर्णय किया जाता है। छोटी पाषाण खान में प्राय: सभी कार्य हाथ से किया जाता है। विस्फोट क्रिया द्वारा चट्टानें तोड़ी जाती हैं। भंगों की अनुपस्थिति में निश्चित दूरी पर खड़े छिद्र बनाए जाते हैं और उनमें विस्फोट किया जाता है। जलज शिलाओं में स्तरों के समांतर क्षैतिज छिद्र बनाकर विस्फोट किया जाता है। साधारणत: खदान सीढ़ीनुमा बनाई जाती है। बहुत बड़ी पाषाण खानों में अधिकाधिक कार्य मशीनों से लिया जाता है।
भारतवर्ष में इमारती पत्थरों के आखनन का कार्य बहुत प्राचीन काल से होता रहा है। दक्षिण भारत के ग्रैनाइट आदि पत्थरों से बने प्रागैतिहासिक काल के मंदिर अभी तक विद्यमान है। आंध्र तथा कर्नाटक राज्यों में इस प्रकार के पत्थरों की खदानें आजकल भी हैं। इनसे पत्थर निकालकर विदेशों को भेजे जाते हैं। महाराष्ट्र और आसपास के क्षेंत्रों में बैसाल्ट अथवा ट्रैप नामक लावा की शिलाओं का प्रयोग इमारती पत्थरों के रूप में किया जाता है। अजंता तथा एलोरा की गुफाएँ इन्हीं पत्थरों में खोदी गई हैं। विंध्य श्रेणी के बलुआ पत्थर दीर्घ काल से हमारी मूल्यवान् निधि रहे हैं। गंगा और यमुना के किनारे खड़े विशाल घाट तथा मंदिर ही नहीं वरन् अनेक प्राचीन अशोकस्तंभ भी इन्हीं से निर्मित हुए हैं। इन पत्थरों की मुख्य खदान कैमूर, चुनार, भरतपुर, फतेहपुर सीकरी आदि स्थानों में स्थित हैं। समस्त उत्तर भारत में अशोककाल से लेकर आज तक इमारती पत्थरों में विंध्य श्रेणी के बलुआ पत्थरों का योगदान सबसे अधिक रहा है।
गोंडवाना युग के बलुआ पत्थर बिहार, उड़ीसा एवं मध्यप्रदेश में तथा महासरट युग के पत्थर कच्छ में निकाले जाते हैं। कायांतरित बलुआ पत्थरों की शिलाएँ अलवर तथा अजमेर में खोदी जाती हैं। सौराष्ट्र में कई स्थानों पर पाषाण खानें हैं, इनमें 'पोरबंदर पत्थर' की खान सबसे मुख्य है। बीजापुर, वारंगल, बूँदी, उदयपुर, मध्य प्रदेश, आंध्र तथा तमिलनाडु राज्यों में भी इस प्रकार के पत्थर निकाले जाते हैं। स्लेट की खदानें कुमायुँ, गढ़वाल, मंडी, चंबा, काँगड़ा आदि पर्वतीय प्रदेशों में बहुलता से मिलती हैं। आंध्र के कुर्नूल जिले में भी स्लेट शिलाएँ अत्यधिक मात्रा में विद्यमान हैं। रेवाड़ी तथा गुड़गाँव में भी स्लेट मिलती है।
संगमरमर शिलाओं के लिए जोधपुर के निकट मकराना की पाषाण खानें दीर्घकाल से प्रसिद्ध हैं। आगरे का ताजमहल एवं कलकत्ते का विक्टोरिया मेमोरियल मकराना संगमरमर का ही बना है। राजस्थान में अलवर, जयपुर, नाथद्वारा, राजनगर, रामालो, आदि संगमरमर के अन्य प्रसिद्ध क्षेत्र हैं। दक्षिण भारत में चीतलदुर्ग, मैसूर, सेलम और मदुराई जिले तथा मध्य प्रदेश में जबलपुर, छिंदवाड़ा और महाराष्ट्र में नागपुर तथा सिवनी जिले सुंदर संगमरमर के लिए प्रसिद्ध हैं। असाधारण रंग के संगमरमर पत्थरों के लिए गुजरात में हरिकुवा, रेवाकाँठा और साँडारा तथा आंध्र प्रदेश में कुर्नूल, कृष्णा और गुंटूर जिले प्रसद्धि हैं।"""

In [ ]:
o1 = decode(encode(val_text3))

In [ ]:
print(o1)

इमारती पत्थरों को खोदकर निकालने की क्रिया को आखनन कहते हैं। इस स्थान को जहाँ से पत्थर निकाले जाते हैं, खादान या 'पाषाण खान' कहते हैं। पाषाण खान साधारणतया खुले स्थान में ही बनाई जाती है। 
इमारती पत्थरों में ग्रैनाइट, बैसाल्ट, बालू के पत्थर, चूने के पत्थर, स्लेट और संगमरमर मुख्य हैं। ग्रैनाइट शब्द के अंतर्गत साधारणतया हल्के रंग की सभी आग्नेय शिलाएँ मानी जाती हैं। इन शिलाओं की रचना क्वार्ट्ज़, फ़ेल्स्पार, अभ्रक और हॉर्न ब्लेंड नामक खनिजों से होती है। 
बैसाल्ट प्राय: काले रंग की शिलाएँ होती हैं। ये ट्रैप भी कहलाती हैं। इनमें फ़ेल्स्पार और पाइरॉक्सीन खनिजों की प्रचुर मात्रा होती है। इन शिलाओं में कई प्रकार के भंग होते हैं, जिनसे इन्हें खोदने में सुविधा होती है। ये सामान्यत: कड़ी होती हैं। ग्रैनाइट शब्द के अंतर्गत ही 'नाइस' नामक कायांतरित शिलाओं को भी गिन लिया जाता है। अभ्रक आदि खनिज के समांतर तलों में व्यवस्थित होने से इनमें अनेक दुर्बल धरातल बन जाते हैं, जिनके कारण इन्हें खोदने में सुकरता हो जाती है। भंगों की उपस्थिति में इसे और भी सरलता से खोदा जा सकता है। बालुकाश्म एवं चूने का पत्थर जलज 